In [1]:
# conda install keras-tuner
import keras_tuner as kt
import pandas as pd
import numpy as np
import keras
from datetime import datetime
from keras.models import Sequential 
from keras.layers import Dense, Dropout
from keras.wrappers.scikit_learn import KerasRegressor, KerasClassifier
from keras import losses
from keras.constraints import maxnorm
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV, cross_validate
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import GradientBoostingRegressor, RandomForestRegressor
import seaborn as sns
sns.set()
from collections import Counter
from scipy.stats import uniform
from scipy.stats import randint
from sklearn.metrics import mean_squared_error, make_scorer, accuracy_score, f1_score, classification_report
# import pickle
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report, roc_auc_score
from keras.models import load_model
import matplotlib.pyplot as plt
from keras.callbacks import *


2022-12-03 10:40:34.241275: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
def mean_absolute_percentage_error(y_true, y_pred): 
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

### Bitcoin

In [3]:
regs = pd.read_csv("../Data/train_btc_selected_features.csv")
btc = pd.read_csv("../Data/btc_Data.csv")
for i in range(len(btc['Date'])):
    btc['Date'][i]  =  datetime.strptime(btc['Date'][i], '%Y/%m/%d')

btc = btc.set_index("Date")

/tmp/ipykernel_452226/2421463472.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btc['Date'][i]  =  datetime.strptime(btc['Date'][i], '%Y/%m/%d')


In [4]:
btcData = btc[regs.columns]

In [5]:
btcData['returns'] = btcData['priceUSD'].pct_change()

/tmp/ipykernel_452226/751970969.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  btcData['returns'] = btcData['priceUSD'].pct_change()


In [6]:
Data = btcData.drop(columns=['priceUSD'])
Data = Data[1:]

In [7]:
Data 

,difficulty14mom,difficulty30mom,difficulty3rsi,difficulty90mom,difficulty90sma,hashrate90std,hashrate90var,median_transaction_fee90rocUSD,median_transaction_feeUSD,mining_profitability,...,price90wmaUSD,sentinusd90emaUSD,sentinusdUSD,top100cap,top100cap14trx,top100cap30rsi,top100cap30trx,top100cap90rsi,top100cap90trx,returns
Date,,,,,,,,,,,,,,,,,,,,,
2010-07-18,1.621380e+02,1.641620e+02,100.00,1.737230e+02,2.055500e+01,8.384389e+08,1.757449e+17,0.000,0.000,401834.0000,...,0.0,0,2.612000e+03,25.782,0.139,71.191,0.308,79.756,0.550,0.466667
2010-07-19,1.621380e+02,1.641620e+02,100.00,1.737230e+02,2.248600e+01,8.819952e+08,1.944789e+17,0.000,0.000,481473.0000,...,0.0,0,4.047000e+03,25.685,0.123,68.863,0.300,78.999,0.547,0.183196
2010-07-20,1.584060e+02,1.641620e+02,100.00,1.735570e+02,2.441400e+01,9.280542e+08,2.153212e+17,0.000,0.000,431831.0000,...,0.0,0,2.341000e+03,25.602,0.107,66.923,0.292,78.355,0.544,-0.088475
2010-07-21,1.580420e+02,1.641620e+02,100.00,1.700790e+02,2.630400e+01,9.761949e+08,2.382391e+17,0.000,0.000,460783.0000,...,0.0,0,2.122000e+03,25.638,0.092,67.334,0.284,78.431,0.541,-0.020434
2010-07-22,1.580420e+02,1.641620e+02,100.00,1.700790e+02,2.819400e+01,1.019474e+09,2.598318e+17,0.000,0.000,334641.0000,...,0.0,0,2.570000e+03,25.604,0.078,66.526,0.275,78.166,0.537,-0.153846
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-11-24,1.882952e+11,1.148115e+11,99.86,8.598887e+12,3.384949e+13,4.612156e+19,5.317997e+38,165.380,0.818,0.0577,...,18820.0,28544682257,6.114577e+09,15.236,-0.080,40.118,-0.042,45.140,0.001,0.008075
2022-11-25,1.882952e+11,1.148115e+11,99.86,8.598887e+12,3.394503e+13,4.542600e+19,5.158804e+38,214.464,0.812,0.0541,...,18759.0,28076113671,7.224812e+09,15.231,-0.075,39.880,-0.043,45.047,0.001,-0.006565
2022-11-26,1.882952e+11,1.148115e+11,99.86,8.598887e+12,3.404058e+13,4.624135e+19,5.345655e+38,255.004,0.809,0.0571,...,18701.0,27566714445,4.898449e+09,15.233,-0.070,40.069,-0.043,45.106,0.001,0.004001


In [8]:
# divide X and Y
X = Data.iloc[:,0:]
#Y = Data['returns']   # 用returns的话就用这一行，然后把下一行comment掉
Y = btcData['priceUSD'].shift(-30)[1:] # 反之亦然

In [9]:
X.head(3)

,difficulty14mom,difficulty30mom,difficulty3rsi,difficulty90mom,difficulty90sma,hashrate90std,hashrate90var,median_transaction_fee90rocUSD,median_transaction_feeUSD,mining_profitability,...,price90wmaUSD,sentinusd90emaUSD,sentinusdUSD,top100cap,top100cap14trx,top100cap30rsi,top100cap30trx,top100cap90rsi,top100cap90trx,returns
Date,,,,,,,,,,,,,,,,,,,,,
2010-07-18,162.138,164.162,100.0,173.723,20.555,838438881.0,1.757449e+17,0.0,0.0,401834.0,...,0.0,0,2612.0,25.782,0.139,71.191,0.308,79.756,0.550,0.466667
2010-07-19,162.138,164.162,100.0,173.723,22.486,881995244.0,1.944789e+17,0.0,0.0,481473.0,...,0.0,0,4047.0,25.685,0.123,68.863,0.300,78.999,0.547,0.183196
2010-07-20,158.406,164.162,100.0,173.557,24.414,928054231.0,2.153212e+17,0.0,0.0,431831.0,...,0.0,0,2341.0,25.602,0.107,66.923,0.292,78.355,0.544,-0.088475


In [10]:
# Split into three data sets
X_train = X['2016-01-01':'2019-12-31']
X_val = X['2020-01-01':'2021-05-31']
X_test = X['2021-06-01':'2023-01-01']

Y_train = Y['2016-01-01':'2019-12-31']
Y_val = Y['2020-01-01':'2021-05-31']
Y_test = Y['2021-06-01':'2023-01-01']

In [11]:
estimators=[]
estimators.append(['mixmax',MinMaxScaler()])
estimators.append(['robust',RobustScaler()])
scale=Pipeline(estimators,verbose=True)
scale.fit(X_train)

[Pipeline] ............ (step 1 of 2) Processing mixmax, total=   0.0s
[Pipeline] ............ (step 2 of 2) Processing robust, total=   0.0s


Pipeline(steps=[('mixmax', MinMaxScaler()), ['robust', RobustScaler()]],
         verbose=True)

In [12]:
X_train=scale.transform(X_train)
X_test=scale.transform(X_test)
X_val = scale.transform(X_val)

In [13]:
def lr_schedule(epoch):
    """Learning Rate Schedule
    # Arguments
        epoch (int): The number of epochs
    # Returns
        lr (float32): learning rate
    """
    lr = 1e-3
    if epoch > 180:
        lr *= 0.5e-3
    elif epoch > 160:
        lr *= 1e-3
    elif epoch > 120:
        lr *= 1e-2
    elif epoch > 80:
        lr *= 1e-1
    print('Learning rate: ', lr)
    return lr

In [14]:
# define neural network model
shape=X.shape[1]
def build_model(hp, initializer='normal', activation='relu', NUM_FEATURES=shape):
    # create model
    hp_units1 = hp.Int('units1', min_value=32, max_value=512, step=32)
    hp_units2 = hp.Int('units2', min_value=32, max_value=512, step=32)
    hp_units3 = hp.Int('units3', min_value=32, max_value=512, step=32)
    model = Sequential()
    model.add(Dense(hp_units1, input_shape=(NUM_FEATURES,), kernel_initializer=initializer, activation=activation))
    model.add(Dense(hp_units2, activation=activation))
    model.add(Dense(hp_units3, activation=activation))
    model.add(Dense(1, activation=activation))
    # Compile model
    adam=keras.optimizers.Adam(lr=lr_schedule(0), amsgrad=True)
    #sgd=keras.optimizers.SGD(learning_rate=0.08, momentum=0.9, nesterov=False)
    model.compile(loss='logcosh', optimizer=adam, metrics=['mae'])
    return model

In [15]:
tuner = kt.RandomSearch(
    build_model,
    objective='val_mae',
    directory='btc_tune',
    project_name='ANN_TUNE'
)

tuner.search(X_train, Y_train, epochs=5000, validation_data=(X_val, Y_val))

INFO:tensorflow:Reloading Oracle from existing project btc_tune/ANN_TUNE/oracle.json
Learning rate:  0.001
INFO:tensorflow:Reloading Tuner from btc_tune/ANN_TUNE/tuner0.json
INFO:tensorflow:Oracle triggered exit


2022-12-03 10:40:37.151046: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-03 10:40:37.151564: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.
/home/spectre/anaconda3/envs/tensorplustorch/lib/python3.10/site-packages/keras/optimizers/optimizer_v2/adam.py:114: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


In [16]:
# Get the best hyperparameters.
best_hp = tuner.get_best_hyperparameters()[0]
# Build the model with the best hp.
regressor = build_model(best_hp)
# Fit with the entire dataset.
X_all = np.concatenate((X_train, X_val))
Y_all = np.concatenate((Y_train, Y_val))


Learning rate:  0.001


In [17]:
# best hyperparameters
best_hp

val_loss is the value of cost function for your cross-validation data and loss is the value of cost function for your training data.

stop training when val_loss is not increasing

In [18]:
earlyStopping = EarlyStopping(monitor='val_loss', patience=1000,verbose=1, mode='auto')

In [19]:
regressor.fit(x=X_all, y=Y_all, epochs=5000, use_multiprocessing=True, callbacks=[earlyStopping])

Epoch 1/5000
62/62 [==============================] - 0s 3ms/step - loss: 9063.7988 - mae: 9064.4922
Epoch 2/5000
62/62 [==============================] - 1s 10ms/step - loss: 4422.3208 - mae: 4423.0142
Epoch 3/5000
62/62 [==============================] - 0s 4ms/step - loss: 2649.4971 - mae: 2650.1895
Epoch 4/5000
62/62 [==============================] - 0s 4ms/step - loss: 2535.3489 - mae: 2536.0410
Epoch 5/5000
62/62 [==============================] - 0s 4ms/step - loss: 2425.7878 - mae: 2426.4807
Epoch 6/5000
62/62 [==============================] - 0s 3ms/step - loss: 2344.9055 - mae: 2345.5989
Epoch 7/5000
62/62 [==============================] - 0s 5ms/step - loss: 2262.5774 - mae: 2263.2700
Epoch 8/5000
62/62 [==============================] - 0s 3ms/step - loss: 2193.7239 - mae: 2194.4167
Epoch 9/5000
62/62 [==============================] - 1s 15ms/step - loss: 2157.3042 - mae: 2157.9976
Epoch 10/5000
62/62 [==============================] - 0s 5ms/step - loss: 2121.3291 - ma

62/62 [==============================] - 0s 6ms/step - loss: 1548.3899 - mae: 1549.0822
Epoch 28/5000
62/62 [==============================] - 0s 5ms/step - loss: 1500.0408 - mae: 1500.7340
Epoch 29/5000
62/62 [==============================] - 0s 4ms/step - loss: 1464.4213 - mae: 1465.1134
Epoch 30/5000
62/62 [==============================] - 0s 6ms/step - loss: 1473.8336 - mae: 1474.5259
Epoch 31/5000
62/62 [==============================] - 0s 4ms/step - loss: 1448.5730 - mae: 1449.2654
Epoch 32/5000
62/62 [==============================] - 0s 3ms/step - loss: 1393.7844 - mae: 1394.4764
Epoch 33/5000
62/62 [==============================] - 0s 3ms/step - loss: 1402.9559 - mae: 1403.6476
Epoch 34/5000
62/62 [==============================] - 0s 4ms/step - loss: 1338.1067 - mae: 1338.7992
Epoch 35/5000
62/62 [==============================] - 0s 3ms/step - loss: 1350.5071 - mae: 1351.1992
Epoch 36/5000
62/62 [==============================] - 0s 4ms/step - loss: 1280.2855 - mae: 1280

62/62 [==============================] - 0s 4ms/step - loss: 998.1720 - mae: 998.8646
Epoch 55/5000
62/62 [==============================] - 0s 5ms/step - loss: 990.0247 - mae: 990.7168
Epoch 56/5000
62/62 [==============================] - 0s 6ms/step - loss: 969.0015 - mae: 969.6929
Epoch 57/5000
62/62 [==============================] - 0s 7ms/step - loss: 949.6632 - mae: 950.3540
Epoch 58/5000
62/62 [==============================] - 0s 5ms/step - loss: 974.5601 - mae: 975.2518
Epoch 59/5000
62/62 [==============================] - 0s 7ms/step - loss: 921.6586 - mae: 922.3503
Epoch 60/5000
62/62 [==============================] - 1s 11ms/step - loss: 909.3143 - mae: 910.0051
Epoch 61/5000
62/62 [==============================] - 1s 9ms/step - loss: 928.1576 - mae: 928.8489
Epoch 62/5000
62/62 [==============================] - 0s 6ms/step - loss: 937.6387 - mae: 938.3302
Epoch 63/5000
62/62 [==============================] - 0s 3ms/step - loss: 883.2151 - mae: 883.9074
Epoch 64/5000

62/62 [==============================] - 0s 6ms/step - loss: 753.2357 - mae: 753.9271
Epoch 82/5000
62/62 [==============================] - 0s 5ms/step - loss: 777.7490 - mae: 778.4411
Epoch 83/5000
62/62 [==============================] - 0s 4ms/step - loss: 767.3788 - mae: 768.0699
Epoch 84/5000
62/62 [==============================] - 0s 6ms/step - loss: 731.9775 - mae: 732.6689
Epoch 85/5000
62/62 [==============================] - 0s 4ms/step - loss: 727.4891 - mae: 728.1801
Epoch 86/5000
62/62 [==============================] - 0s 3ms/step - loss: 713.6987 - mae: 714.3896
Epoch 87/5000
62/62 [==============================] - 0s 5ms/step - loss: 751.4442 - mae: 752.1352
Epoch 88/5000
62/62 [==============================] - 0s 6ms/step - loss: 723.2495 - mae: 723.9421
Epoch 89/5000
62/62 [==============================] - 0s 5ms/step - loss: 733.2668 - mae: 733.9581
Epoch 90/5000
62/62 [==============================] - 1s 13ms/step - loss: 722.4156 - mae: 723.1076
Epoch 91/5000

62/62 [==============================] - 0s 3ms/step - loss: 651.8675 - mae: 652.5591
Epoch 109/5000
62/62 [==============================] - 0s 7ms/step - loss: 650.0786 - mae: 650.7704
Epoch 110/5000
62/62 [==============================] - 0s 6ms/step - loss: 686.9119 - mae: 687.6039
Epoch 111/5000
62/62 [==============================] - 0s 6ms/step - loss: 663.2028 - mae: 663.8948
Epoch 112/5000
62/62 [==============================] - 0s 4ms/step - loss: 623.8024 - mae: 624.4948
Epoch 113/5000
62/62 [==============================] - 0s 6ms/step - loss: 631.5778 - mae: 632.2684
Epoch 114/5000
62/62 [==============================] - 0s 5ms/step - loss: 649.1298 - mae: 649.8210
Epoch 115/5000
62/62 [==============================] - 0s 5ms/step - loss: 624.2217 - mae: 624.9125
Epoch 116/5000
62/62 [==============================] - 0s 7ms/step - loss: 622.8934 - mae: 623.5846
Epoch 117/5000
62/62 [==============================] - 0s 5ms/step - loss: 658.1060 - mae: 658.7983
Epoch

...

62/62 [==============================] - 1s 10ms/step - loss: 261.2130 - mae: 261.9027
Epoch 1054/5000
62/62 [==============================] - 0s 6ms/step - loss: 274.8681 - mae: 275.5584
Epoch 1055/5000
62/62 [==============================] - 0s 3ms/step - loss: 272.4764 - mae: 273.1659
Epoch 1056/5000
62/62 [==============================] - 1s 9ms/step - loss: 262.5859 - mae: 263.2738
Epoch 1057/5000
62/62 [==============================] - 0s 5ms/step - loss: 278.2716 - mae: 278.9625
Epoch 1058/5000
62/62 [==============================] - 0s 5ms/step - loss: 273.2473 - mae: 273.9363
Epoch 1059/5000
62/62 [==============================] - 1s 12ms/step - loss: 267.3084 - mae: 267.9959
Epoch 1060/5000
62/62 [==============================] - 0s 6ms/step - loss: 276.5078 - mae: 277.1981
Epoch 1061/5000
62/62 [==============================] - 0s 5ms/step - loss: 293.3392 - mae: 294.0285
Epoch 1062/5000
62/62 [==============================] - 0s 4ms/step - loss: 270.0447 - mae: 270

62/62 [==============================] - 0s 4ms/step - loss: 269.6994 - mae: 270.3890
Epoch 1081/5000
62/62 [==============================] - 0s 4ms/step - loss: 291.9992 - mae: 292.6889
Epoch 1082/5000
62/62 [==============================] - 0s 4ms/step - loss: 262.7920 - mae: 263.4820
Epoch 1083/5000
62/62 [==============================] - 0s 3ms/step - loss: 264.8812 - mae: 265.5700
Epoch 1084/5000
62/62 [==============================] - 0s 3ms/step - loss: 245.8120 - mae: 246.5019
Epoch 1085/5000
62/62 [==============================] - 0s 5ms/step - loss: 282.6929 - mae: 283.3823
Epoch 1086/5000
62/62 [==============================] - 0s 7ms/step - loss: 276.6182 - mae: 277.3076
Epoch 1087/5000
62/62 [==============================] - 0s 3ms/step - loss: 261.4912 - mae: 262.1821
Epoch 1088/5000
62/62 [==============================] - 0s 3ms/step - loss: 263.7001 - mae: 264.3898
Epoch 1089/5000
62/62 [==============================] - 0s 6ms/step - loss: 257.1518 - mae: 257.8

62/62 [==============================] - 1s 9ms/step - loss: 254.0862 - mae: 254.7747
Epoch 1108/5000
62/62 [==============================] - 1s 10ms/step - loss: 267.0977 - mae: 267.7856
Epoch 1109/5000
62/62 [==============================] - 0s 4ms/step - loss: 264.2073 - mae: 264.8963
Epoch 1110/5000
62/62 [==============================] - 1s 10ms/step - loss: 244.6443 - mae: 245.3329
Epoch 1111/5000
62/62 [==============================] - 1s 18ms/step - loss: 287.5933 - mae: 288.2833
Epoch 1112/5000
62/62 [==============================] - 0s 5ms/step - loss: 276.4493 - mae: 277.1390
Epoch 1113/5000
62/62 [==============================] - 0s 4ms/step - loss: 274.9512 - mae: 275.6401
Epoch 1114/5000
62/62 [==============================] - 0s 7ms/step - loss: 270.6331 - mae: 271.3209
Epoch 1115/5000
62/62 [==============================] - 0s 5ms/step - loss: 264.5991 - mae: 265.2883
Epoch 1116/5000
62/62 [==============================] - 0s 3ms/step - loss: 273.6060 - mae: 27

62/62 [==============================] - 0s 4ms/step - loss: 270.9600 - mae: 271.6506
Epoch 1135/5000
62/62 [==============================] - 0s 5ms/step - loss: 251.8986 - mae: 252.5873
Epoch 1136/5000
62/62 [==============================] - 0s 6ms/step - loss: 250.2251 - mae: 250.9135
Epoch 1137/5000
62/62 [==============================] - 0s 4ms/step - loss: 275.4455 - mae: 276.1361
Epoch 1138/5000
62/62 [==============================] - 0s 7ms/step - loss: 269.4072 - mae: 270.0974
Epoch 1139/5000
62/62 [==============================] - 0s 5ms/step - loss: 287.6353 - mae: 288.3256
Epoch 1140/5000
62/62 [==============================] - 1s 11ms/step - loss: 274.8063 - mae: 275.4962
Epoch 1141/5000
62/62 [==============================] - 0s 6ms/step - loss: 278.8995 - mae: 279.5893
Epoch 1142/5000
62/62 [==============================] - 0s 5ms/step - loss: 268.2366 - mae: 268.9254
Epoch 1143/5000
62/62 [==============================] - 1s 9ms/step - loss: 277.3697 - mae: 278.

KeyboardInterrupt: 

In [ ]:

y_pred=regressor.predict(X_test)

In [ ]:
# for check
Y_train_pred=regressor.predict(X_train)
r2_score(Y_train, Y_train_pred) #training score

In [ ]:
r2=r2_score(Y_test[:-30],y_pred[:-30]) #score/ r^2
print(f'r2:{r2}')

In [ ]:
# r2_oos
def r2_oos(ret, pred):
    sum_of_sq_res = np.nansum(np.power((ret-pred), 2))
    sum_of_sq_total = np.nansum(np.power(ret, 2))
    
    return 1-sum_of_sq_res/sum_of_sq_total

In [ ]:
r2_oos = r2_oos(Y_test[:-30], y_pred[:-30])
print(f'r2_oos:{r2_oos}')

In [ ]:
mae=mean_absolute_error(Y_test[:-30],y_pred[:-30]) #mae
print(f'mae:{mae}')

rmse=np.sqrt(mean_squared_error(Y_test[:-30],y_pred[:-30])) #rmse
print(f'rmse:{rmse}')

mape=mean_absolute_percentage_error(Y_test[:-30],y_pred[:-30]) #mape
print(f'mape:{mape}')

-----------------------------

In [ ]:
pre_df = pd.DataFrame(zip(Y_test,y_pred),columns=['Y_test','y_pred'])
pre_df.index = Y_test.index
pre_df

In [ ]:
pre_df['pred_returns'] = pre_df['y_pred'].pct_change()

In [ ]:
pre_df

In [ ]:
pd.DataFrame(zip(Y_test,y_pred),columns=['Y_test','y_pred']).plot()

In [ ]:
pre_df.to_csv("../result/ANN/btc_NN.csv")

In [ ]:
!kdeconnect-cli -n TAS-AN00 --ping-msg 'Script complete!'